# Applying data minimization to a trained ML model

In this tutorial we will show how to perform data minimization for ML models using the minimization module. 

This will be demonstarted using the Adult dataset (original dataset can be found here: https://archive.ics.uci.edu/ml/datasets/adult). 

We use only the numerical features in the dataset because this is what is currently supported by the module.

Furthermore, We will explore how to use the anonymization features to further increase the data protection capabilities of the solution

## Load data

In [1]:
import numpy as np
import pandas as pd
import warnings
import anonymize_module as an
import os
import sys
import calcmetric as cm
sys.path.insert(0, os.path.abspath('..'))
from apt.utils.datasets import ArrayDataset
from apt.utils.models import SklearnClassifier, ModelOutputType
from sklearn.tree import DecisionTreeClassifier

warnings.filterwarnings('ignore', category=UserWarning)
warnings.filterwarnings('ignore', category=FutureWarning)
# Use only numeric features (age, education-num, capital-gain, capital-loss, hours-per-week)


x_train = np.loadtxt("https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data",
                        usecols=(0, 4, 10, 11, 12), delimiter=",")

y_train = np.loadtxt("https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data",
                        usecols=14, dtype=str, delimiter=",")


x_test = np.loadtxt("https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.test",
                        usecols=(0, 4, 10, 11, 12), delimiter=",", skiprows=1)

y_test = np.loadtxt("https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.test",
                        usecols=14, dtype=str, delimiter=",", skiprows=1)

x_train = x_train[:1500]
y_train = y_train[:1500]
x_test = x_test[:1500]
y_test = y_test[:1500]

# Trim trailing period "." from label
y_test = np.array([a[:-1] for a in y_test])
y_train[y_train == ' <=50K'] = 0
y_train[y_train == ' >50K'] = 1
y_train = y_train.astype(int)

y_test[y_test == ' <=50K'] = 0
y_test[y_test == ' >50K'] = 1
y_test = y_test.astype(int)


## Train decision tree model

In [2]:
import os
import sys
sys.path.insert(0, os.path.abspath('..'))

from apt.utils.datasets import ArrayDataset
from apt.utils.models import SklearnClassifier, ModelOutputType
from sklearn.tree import DecisionTreeClassifier

base_est = DecisionTreeClassifier()
model = SklearnClassifier(base_est, ModelOutputType.CLASSIFIER_PROBABILITIES)
model.fit(ArrayDataset(x_train, y_train))

print('Base model accuracy: ', model.score(ArrayDataset(x_test, y_test)))

Base model accuracy:  0.766


## Run minimization
We will try to run minimization with different possible values of target accuracy (how close to the original model's accuracy we want to get, 1 being same accuracy as for original data).

In [3]:
from apt.minimization import GeneralizeToRepresentative
from sklearn.model_selection import train_test_split

# default target_accuracy is 0.998
minimizer = GeneralizeToRepresentative(model)

# Fitting the minimizar can be done either on training or test data. Doing it with test data is better as the 
# resulting accuracy on test data will be closer to the desired target accuracy (when working with training 
# data it could result in a larger gap)
# Don't forget to leave a hold-out set for final validation!
X_generalizer_train, x_test, y_generalizer_train, y_test = train_test_split(x_test, y_test, stratify=y_test,
                                                                test_size = 0.4, random_state = 38)
x_train_predictions = model.predict(ArrayDataset(X_generalizer_train))
if x_train_predictions.shape[1] > 1:
    x_train_predictions = np.argmax(x_train_predictions, axis=1)
minimizer.fit(dataset=ArrayDataset(X_generalizer_train, x_train_predictions))
transformed = minimizer.transform(dataset=ArrayDataset(x_test))

print('Accuracy on minimized data: ', model.score(ArrayDataset(transformed, y_test)))

Initial accuracy of model on generalized data, relative to original model predictions (base generalization derived from tree, before improvements): 0.838889
Improving accuracy
feature to remove: 2
Removed feature: 2, new relative accuracy: 0.841667
feature to remove: 3
Removed feature: 3, new relative accuracy: 0.841667
feature to remove: 0
Removed feature: 0, new relative accuracy: 0.883333
feature to remove: 1
Removed feature: 1, new relative accuracy: 0.952778
feature to remove: 4
Removed feature: 4, new relative accuracy: 1.000000
Accuracy on minimized data:  0.7833333333333333


#### Let's see what features were generalized

In [4]:
generalizations = minimizer.generalizations
print(generalizations)

{'ranges': {}, 'categories': {}, 'untouched': ['3', '1', '0', '2', '4'], 'category_representatives': {}, 'range_representatives': {}}


We can see that for the default target accuracy of 0.998 of the original accuracy, no generalizations are possible (all features are left untouched, i.e., not generalized).

Let's change to a slightly lower target accuracy.

In [5]:
# We allow a 10% deviation in accuracy from the original model accuracy
minimizer2 = GeneralizeToRepresentative(model, target_accuracy=0.9)

minimizer2.fit(dataset=ArrayDataset(X_generalizer_train, x_train_predictions))
transformed2 = minimizer2.transform(dataset=ArrayDataset(x_test))
print('Accuracy on minimized data: ', model.score(test_data=ArrayDataset(transformed2, y_test)))
generalizations2 = minimizer2.generalizations
print(generalizations2)

Initial accuracy of model on generalized data, relative to original model predictions (base generalization derived from tree, before improvements): 0.838889
Improving accuracy
feature to remove: 2
Removed feature: 2, new relative accuracy: 0.841667
feature to remove: 3
Removed feature: 3, new relative accuracy: 0.841667
feature to remove: 0
Removed feature: 0, new relative accuracy: 0.883333
feature to remove: 1
Removed feature: 1, new relative accuracy: 0.952778
Accuracy on minimized data:  0.7683333333333333
{'ranges': {'4': [18.0, 21.5, 22.5, 30.0, 35.0, 37.5, 38.0, 40.5, 41.0, 43.5, 46.5, 47.0, 47.5, 49.0, 51.0, 52.5, 57.5, 62.5, 65.0, 82.0]}, 'categories': {}, 'untouched': ['3', '2', '1', '0'], 'category_representatives': {}, 'range_representatives': {'4': [18.0, 12.0, 20.0, 22.0, 28.0, 35.0, 36.0, 38.0, 40.0, 1.25, 42.0, 45.0, 0.25, 0.75, 48.0, 50.0, 52.0, 55.0, 60.0, 8.5]}}


## Construct the dataset and measure k-anonymity
dataset consists of the transformed data and the sensitive column

In [6]:
features = ["age", "education-num", "capital-gain", "capital-loss", "hours-per-week"]
QI = ["age", "capital-gain", "capital-loss"]
sensitive_column = 'income'

df = pd.DataFrame(transformed2, columns=features)
df[sensitive_column] = y_test

# find maximum k-value for which k-anonymity is still satisfied
cm.find_k_anonymity(df, QI)

Dataset satisfies 1-anonymity
Dataset does not satisfy 2-anonymity
Dataset satisfies maximum 1-anonymity


1

## K-Anonymize
As seen, the dataset is not anonymized
To anonymize the dataset, call k-anonymize function

In [7]:
k_anonymdf = an.anonymize_k_anonymity(df, QI, sensitive_column, k=10)

=========Start anonymization process:
Partition the dataset:
31 partitions created.
Changing quasi-identifiers values with the aggregation of their partition
Showcasing some of the anonymized partition:
Partition 7
+-----+-------+-----------------+----------------+----------------+------------------+----------+
|     |   age |   education-num |   capital-gain |   capital-loss |   hours-per-week |   income |
|-----+-------+-----------------+----------------+----------------+------------------+----------|
|   8 |    23 |              10 |              0 |              0 |               40 |        0 |
|  16 |    23 |              13 |              0 |              0 |               43 |        0 |
|  22 |    23 |              10 |              0 |              0 |               40 |        0 |
|  24 |    23 |              10 |              0 |              0 |               40 |        1 |
| 145 |    23 |               9 |              0 |              0 |               40 |        0 |
|

Let's check again

In [8]:
cm.find_k_anonymity(k_anonymdf, QI)

Dataset satisfies 1-anonymity
Dataset satisfies 2-anonymity
Dataset satisfies 3-anonymity
Dataset satisfies 4-anonymity
Dataset satisfies 5-anonymity
Dataset satisfies 6-anonymity
Dataset satisfies 7-anonymity
Dataset satisfies 8-anonymity
Dataset satisfies 9-anonymity
Dataset satisfies 10-anonymity
Dataset does not satisfy 11-anonymity
Dataset satisfies maximum 10-anonymity


10

Now, the dataset is 10-anonymous.

## L-diversity

In [9]:
cm.find_l_diversity(k_anonymdf, QI, sensitive_column)
l_diversedf = an.anonymize_l_diversity(df, QI, sensitive_column, 10, 2)
cm.find_l_diversity(l_diversedf, QI, sensitive_column)

Dataset satisfies 1-diversity
Dataset does not satisfy 2-diversity
Dataset satisfies maximum 1-diversity
=========Start anonymization process:
Partition the dataset:
25 partitions created.
Changing quasi-identifiers values with the aggregation of their partition
Showcasing some of the anonymized partition:
Partition 4
+-----+-------+-----------------+----------------+----------------+------------------+----------+
|     |   age |   education-num |   capital-gain |   capital-loss |   hours-per-week |   income |
|-----+-------+-----------------+----------------+----------------+------------------+----------|
|  32 |    29 |              10 |        1298.25 |            154 |               40 |        1 |
|  34 |    29 |              13 |        1298.25 |            154 |               40 |        1 |
|  51 |    29 |              13 |        1298.25 |            154 |               50 |        1 |
| 169 |    29 |               7 |        1298.25 |            154 |               40 |      

2

## T-closeness
Tries to create partitions so that the distribution of the sensitive column is similar to the entire dataset

In [10]:
t_closedf = an.anonymize_t_closeness(df, QI, sensitive_column, k = 8, t=1)


=========Start anonymization process:
Partition the dataset:
33 partitions created.
Changing quasi-identifiers values with the aggregation of their partition
Showcasing some of the anonymized partition:
Partition 16
+-----+-------+-----------------+----------------+----------------+------------------+----------+
|     |   age |   education-num |   capital-gain |   capital-loss |   hours-per-week |   income |
|-----+-------+-----------------+----------------+----------------+------------------+----------|
|   3 |    42 |               9 |        851.688 |              0 |               45 |        1 |
|  26 |    42 |              13 |        851.688 |              0 |               40 |        0 |
|  74 |    42 |               9 |        851.688 |              0 |               45 |        1 |
|  79 |    42 |              10 |        851.688 |              0 |               48 |        1 |
| 101 |    42 |               9 |        851.688 |              0 |               45 |        0 |


## Using the anonymized data as input to ML model

In [11]:
print('Accuracy on k_anonymized/minimized data: ', model.score(test_data=ArrayDataset(k_anonymdf[features], y_test)))
print('Accuracy on l_diversed/minimized data: ', model.score(test_data=ArrayDataset(l_diversedf[features], y_test)))
print('Accuracy on t_close/minimized data: ', model.score(test_data=ArrayDataset(t_closedf[features], y_test)))


Accuracy on k_anonymized/minimized data:  0.715
Accuracy on l_diversed/minimized data:  0.715
Accuracy on t_close/minimized data:  0.7116666666666667
